In [47]:
import pandas as pd
import os
exp_name = 'deepfilternet3'
path = f'../csv_results/{exp_name}/{exp_name}_results.csv'
df = pd.read_csv(path)

In [48]:
df.describe()

,SNR,EER
count,90.000000,90.000000
mean,10.000000,0.072130
std,7.110682,0.054866
min,0.000000,0.014893
25%,5.000000,0.034802
50%,10.000000,0.050181
75%,15.000000,0.092581
max,20.000000,0.257530


In [49]:
df.head(30)

,Noise,SNR,Model,EER
0,AirConditioner,0,campplus,0.090241
1,AirConditioner,0,ecapa_tdnn,0.089676
2,AirConditioner,0,redimnet,0.077031
3,AirConditioner,10,campplus,0.046449
4,AirConditioner,10,ecapa_tdnn,0.049857
5,AirConditioner,10,redimnet,0.037518
6,AirConditioner,15,campplus,0.035269
7,AirConditioner,15,ecapa_tdnn,0.039137
8,AirConditioner,15,redimnet,0.028633
9,AirConditioner,20,campplus,0.028282


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Noise   90 non-null     object 
 1   SNR     90 non-null     int64  
 2   Model   90 non-null     object 
 3   EER     90 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.9+ KB


In [51]:
# Use existing DataFrame 'df' and matplotlib which is already imported
from matplotlib import pyplot as plt


# Define model name mapping
model_mapping = {
    'ecapa_tdnn': 'ECAPA',
    'campplus': 'CAM++',
    'redimnet': 'REDIMNET'
}

model_color_mapping = {
    'ecapa_tdnn':  'blue',
    'campplus': 'orange',
    'redimnet': 'green'
}
noise_types = df['Noise'].unique()
noise_mapping = {
    'AirConditioner': 'Air Conditioner',
    'Babble': 'Babble',
    'Neighbor': 'Neighbour',
    'noisy_gaussian': 'Gaussian',
    'noisy_poisson': 'Poisson',
    'noisy_rayleigh': 'Rayleigh',
}

max_ylim = 33

In [52]:
# Plot for each noise type
for noise in noise_types:
    plt.figure(figsize=(4, 6), dpi=300)

    # Plot for each model
    for model in model_mapping.keys():
        # Filter data for current noise and model
        mask = (df['Noise'] == noise) & (df['Model'] == model)
        data = df[mask].sort_values('SNR')

        plt.plot(
            data['SNR'],
            data['EER'] * 100,  # Convert to percentage
            marker="o",
            label=model_mapping[model],
            linestyle="-",
            alpha=0.6,
            color=model_color_mapping[model],  
        )

    plt.title(f"EER vs SNR for {noise_mapping[noise]} Noise")
    plt.xlabel("SNR [dB]")
    plt.ylabel("EER [%]")
    plt.ylim(0, max_ylim)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    # plt.show()  # Uncommenting to display the plot

    # save the figure into ../figures/
    figure_path = (
        f"../figures/{exp_name}/{exp_name}_{noise_mapping[noise]}_eer_vs_snr.png"
    )
    os.makedirs(os.path.dirname(figure_path), exist_ok=True)
    plt.savefig(figure_path, dpi=300)
    plt.close()

# Summary

In [53]:
# Set figure size
plt.figure(figsize=(8, 6), dpi=300)

sorted_noise_types = sorted(noise_types, key=lambda x: x)

# Create bar plot with error bars showing quartiles
for i, model in enumerate(model_mapping.keys()):
    model_data = [
        df[(df["Model"] == model) & (df["Noise"] == dist)]["EER"] * 100
        for dist in sorted_noise_types
    ]
    # Plot with slight offset for each model
    positions = [x + i * 0.25 for x in range(len(sorted_noise_types))]
    bp = plt.boxplot(
        model_data,
        positions=positions,
        labels=[""] * len(sorted_noise_types),
        widths=0.2,
        patch_artist=True,
        boxprops=dict(facecolor=model_color_mapping[model], alpha=0.5),
    )
    # Add to legend
    plt.plot([], [], color=model_color_mapping[model], label=model_mapping[model])

# Set x-axis labels using noise_mapping
plt.xticks(
    [x + 0.25 for x in range(len(sorted_noise_types))], 
    [noise_mapping[noise] for noise in sorted_noise_types]
)

plt.xlabel('Noise Types')
plt.ylabel('EER [%]')
plt.title('Results by Model and Noise Type')
plt.ylim(0)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()

# Save the figure into ../figures/
figure_path = f"../figures/{exp_name}/{exp_name}_boxplot.png"
os.makedirs(os.path.dirname(figure_path), exist_ok=True)
plt.savefig(figure_path, dpi=300)
plt.close()

/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
/var/folders/nc/8gf92v353514742zf9ytrs600000gn/T/ipykernel_16441/351460442.py:14: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = plt.boxplot(
